In [19]:
import pandas as pd
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import json
client = OpenAI(api_key="sk-e5041bba86ab45389e42f916e15f80e4", base_url="https://api.deepseek.com")



def filter_(title,abs,key_words,exclude_keywords):
    response = client.chat.completions.create(
    model="deepseek-coder",
    messages=[
        {"role": "system", "content": "You are a helpful paper assistant, always udge whether it meets my requirements."},
        {"role": "user", "content": """ 
         Please carefully read the title and abstract of this paper, and consider how it relates to the keywords I need.

         If the abstract and title of this paper only about my requirement keywords and are not in an area I don't require, give me true.
        
         Please return false or true in `match_result to me, where match_result mean whether macth my requestment.      
         
          Follow the example below:
            User: 
            The title of this paper is: example title
            The abstract of this paper is: example abstract
            My requirement keywords: example keywords
            I don't require: example exclude_keywords
         
            Assistant return: {
            "match_result": false,
            }
         
            """
         +f""" 
            My Input:
            [
            The title of this paper is: {title}
            The abstract of this paper is: {abs}
            My requirement keywords: {key_words}
            I don't require: {exclude_keywords}
            ]
        """},
    ],
    stream=False
    )
    return response.choices[0].message.content
# 读取Excel文件
df = pd.read_excel('eccv24.xlsx')


include_keywords = ['Retrieval-Augmented']  # 替换为实际关键词
exclude_keywords = ['3D','Human Rendering','Diffusion','diffusion']  # 替换为实际关键词
keywords=",".join(include_keywords)
exclude_keyword=",".join(exclude_keywords)
print(f"Total number of rows: {len(df)}")
def filter_row(row):
    resp=filter_(row['title'],row['abstract'],keywords,exclude_keyword)

    return ('True' in resp) or ('true' in resp)
    

filtered_rows = []

# Use ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor(max_workers=64) as executor:
    futures = {executor.submit(filter_row, row): index for index, row in df.iterrows()}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
        index = futures[future]
        try:
            if future.result():
                filtered_rows.append(df.iloc[index][['title', 'abstract']])
        except Exception as e:
            print(f"Error processing row {index}: {e}")

# Create a new DataFrame from the filtered rows
filtered_df = pd.DataFrame(filtered_rows)



filtered_df.to_excel('RAG_eccv2024.xlsx', index=False)

print("数据已成功保存到 RAG_eccv2024.xlsx 文件中。")


Total number of rows: 2387


Processing: 100%|██████████| 2387/2387 [01:00<00:00, 39.19it/s]

数据已成功保存到 RAG_eccv2024.xlsx 文件中。


In [1]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from tqdm import tqdm

import time
import requests
import random
import json
from openai import OpenAI
client = OpenAI(api_key="sk-e5041bba86ab45389e42f916e15f80e4", base_url="https://api.deepseek.com")
def translate(query):
    response = client.chat.completions.create(
    model="deepseek-coder",
    messages=[
        {"role": "system", "content": "You are a helpful translate assistant , always tranlate en to cn"},
        {"role": "user", "content": f"请翻译为中文:{query}"},
    ],
    stream=False
    )
    return response.choices[0].message.content
    # Show response


df_filter = pd.read_excel('icml24.xlsx')
tqdm.pandas()

# Apply the translate function with a progress bar
def parallel_apply(series, func, num_threads=64):
    results = [None] * len(series)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = {executor.submit(func, text): idx for idx, text in enumerate(series)}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
            idx = futures[future]
            try:
                result = future.result()
                results[idx] = result
            except Exception as exc:
                print(f"Generated an exception: {exc}")
                results[idx] = None
                
    return results
# Apply the translate function with multithreading and a progress bar
df_filter['abstract0'] = parallel_apply(df_filter['abstract'], translate)
df_filter['title0'] = parallel_apply(df_filter['title'], translate)
df_filter.to_excel('icml24_translated.xlsx', index=False)

Processing: 100%|██████████| 2610/2610 [00:57<00:00, 45.55it/s]


In [16]:
filtered_df.to_excel('deepseek_coder_cvpr2024.xlsx', index=False)

In [9]:
import pandas as pd
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
client = OpenAI(api_key="sk-e5041bba86ab45389e42f916e15f80e4", base_url="https://api.deepseek.com")
include_keywords = ['Efficie','efficien','RAG','Video',]  # 替换为实际关键词
exclude_keywords = ['3D','Human Rendering','Diffusion','diffusion']  # 替换为实际关键词
keywords=";".join(include_keywords)
exclude_keyword=";".join(exclude_keywords)


def filter_(title,abs,key_words,exclude_keywords):
    response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful paper assistant, always udge whether it meets my requirements."},
        {"role": "user", "content": """ 
            Please judge whether this paper meets my requirements and return the result in the specified format.
            Please return false or true in`match_result to me, where match_result mean whether macth my requestment. Follow the example below:
            User: 
            The title of this paper is: example title
            The abstract of this paper is: example abstract
            My requirement keywords: example keywords
            Do not contain this content: example exclude_keywords
         
            Assistant return: {
            "match_result": false,
            }
         
            """
         +f""" 
            My Input:
            [
            The title of this paper is: {title}
            The abstract of this paper is: {abs}
            My requirement keywords: {key_words}
            Do not contain this content: {exclude_keywords}
            ]
        """},
    ],
    stream=False
    )
    return response.choices[0].message.content

In [7]:
rep=filter_('General Object Foundation Model for Images and Videos at Scale',"We present GLEE in this work an object-level foundation model for locating and identifying objects in images and videos. Through a unified framework GLEEaccomplishes detection segmentation tracking grounding and identification of arbitrary objects in the open world scenario for various object perception tasks. Adopting a cohesive learning strategy GLEE acquires knowledge from diverse data sources with varying supervision levels to formulate general object representations excelling in zero-shot transfer to new data and tasks. Specifically we employ an image encoder text encoder and visual prompter to handle multi-modal inputs enabling to simultaneously solve various object-centric downstream tasks while maintaining state-of-the-art performance. Demonstrated through extensive training on over five million images from diverse benchmarks GLEE exhibits remarkable versatility and improved generalization performance efficiently tackling downstream tasks without the need for task-specific adaptation. By integrating large volumes of automatically labeled data we further enhance its zero-shot generalization capabilities. Additionally GLEE is capable of being integrated into Large Language Models serving as a foundational model to provide universal object-level information for multi-modal tasks. We hope that the versatility and universality of our method will mark a significant step in the development of efficient visual foundation models for AGI systems. The models and code are released at https://github.com/FoundationVision/GLEE.",keywords,exclude_keyword)



In [9]:
rep

'{\n"match_result": true,\n}'

In [5]:
import json
rere=rep.replace('json','')
rere=rere.replace('```','')
rere=rere.replace('\n','')
rere=rere.replace(' ','')
data = json.loads(rere)

In [7]:
data

[{'match_result': True, 'match_score': 8.5}]